In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1
%env ANYWIDGET_DEV=1

env: ANYWIDGET_HMR=1
env: ANYWIDGET_DEV=1


In [2]:
from datasets import load_dataset

from seq import Widget

ds = load_dataset("ajaykarthick/imdb-movie-reviews")["test"]

True


In [3]:
widget = Widget(sequences=[m["review"] for m in ds if m["label"] == 1])

Processing feature elements


In [4]:
widget

Widget(feature_ids=[213, 3, 29, 66, 10, 5, 199, 418, 638, 573], ids=[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 4, 10, 6, …

In [6]:
import numpy as np

ids = np.array(widget.ids)
feature_ids = np.array(widget.feature_ids)

ids.shape, feature_ids.shape

((5000, 32), (10,))

In [16]:
# id that feature_ids contains

print(ids.flatten().shape)
np.isin(ids.flatten(), feature_ids.flatten()).sum()

5000


np.int64(13458)